## Module 2 Final Project Submission
* Name: Vivienne DiFrancesco
* Pace: Full Time
* Instructor: James Irving

# Introduction

The goal of this project 

# Obtaining the data

In [ ]:
# Importing libraries that I will use
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
%matplotlib inline

# Setting default seaborn setting for my visuals
sns.set(style="whitegrid")

# Supressing warnings
import warnings
warnings.filterwarnings('ignore')

# Importing the statsmodels packages I will use
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Importing scikit learn packages I will use
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

In [ ]:
# Setting pandas to display max columns and rows
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', None)

# Turning off scientific notation in pandas
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
# Loading in the data
df = pd.read_csv('kc_house_data.csv')
df.head()

In [ ]:
# Set the index to the id
df.set_index('id', inplace=True)

In [ ]:
# Checking out the length and columns
df.shape

In [ ]:
# Checking the data types and where there might be nulls
df.info()

# Scrubbing the data

## Addressing the price column

I started with the price column since that is the target. I wanted to get to know the data a little using describe(). I looked at value_counts() to make sure there weren't issues with rogue values like 0000 or something that would not register as nulls.

In [ ]:
# Making price an integer instead of a float
df.price = df.price.astype('int64')

In [ ]:
# Checking the stats for the column to see if everything looks normal
df.price.describe()

In [ ]:
# Double checking that there aren't rogue values hiding in the data
df.price.value_counts()[:20]

## Dealing with NA values

I then turned to the other columns to deal with NA values. I filled the NA values, cast them to the correct data type, and then used value_counts() to check for rogue entries that may have been missed.

In [ ]:
# Looking at all NA values in all columns
df.isna().sum()

> I tried mapping the entries that were missing waterfront and it seems as if some of the entries would be waterfront properties. I decided to fill the null values based on the ratio of 0 and 1 that are already in the dataset.

In [ ]:
# Creating a sub-dataframe of the missing entries to use for visualizing
waterfront_check = df.copy()
waterfront_check = waterfront_check[waterfront_check['waterfront'].isna()]

In [ ]:
# Saving the file

# waterfront_check.to_csv(r'C:\Users\drudi\DataScience\Module02\FinalProject\waterfront_check.csv')

This map was created using the waterfront_check dataframe loaded into Tableau Public. This screenshot is a zoomed in view to better see individual entries as an example. The full image can be viewed and downloaded from https://public.tableau.com/profile/vivienne4370 

<img src="waterfrontcheck.png">

In [ ]:
# Checking the percentages of the different values
df.waterfront.value_counts(normalize=True)

In [ ]:
# Checking value counts before filling the missing values
df.waterfront.value_counts()

In [ ]:
# Setting the probability ratios based on the value counts
prob = df.waterfront.value_counts(normalize=True).values

# Setting a seed so that the random results are the same every time
np.random.seed(123)

# Filling the missing values with either 0 or 1 using the probability
df["waterfront"] = df["waterfront"].apply(lambda x: 
                                          np.random.choice([0, 1], p=prob) 
                                          if (np.isnan(x)) else x)

In [ ]:
# Making sure the value counts changed appropriately 
# and there are no nulls left
df["waterfront"].value_counts(normalize=True, dropna=False)

In [ ]:
# Changing the datatype
df.waterfront = df.waterfront.astype('int64')

> I dropped the view column since it is not clear what this data represents. It does not represent the views from the house but likely has something to do with listing views. Without knowing what it could mean, I dropped it to avoid any confusion from the column.

In [ ]:
# Filling NA values with 0
df.drop(columns='view', inplace=True)

> I decided to fill the yr_renovated columns with zeros and just assume that if the value is null that it means the house has not been renovated.

In [ ]:
# Filling NA values with 0
df.yr_renovated.fillna(0, inplace=True)

In [ ]:
# Checking for rogue values
df.yr_renovated.value_counts()

In [ ]:
# Changing the datatype
df.yr_renovated = df.yr_renovated.astype('int64')

In [ ]:
# Verifying that all NAs were dealt with
df.isna().sum()

## Checking for strange values in other columns

I looked through the rest of my columns for rogue entries and to generally better get to know my data.

In [ ]:
# Looking at columns and data types
df.info()

In [ ]:
# Checking for any strange values
df.date.value_counts()[:20]

> There is an entry with 33 bedrooms. I'm going to leave it for now and deal with it later when I do outlier removal.

In [ ]:
# Checking for any strange values
df.bedrooms.value_counts()

In [ ]:
# Checking for any strange values
df.bathrooms.value_counts()

In [ ]:
# Checking for any strange values
df.sqft_living.value_counts()[:20]

In [ ]:
# Checking for any strange values
df.sqft_lot.value_counts()[:20]

In [ ]:
# Checking for any strange values
df.floors.value_counts()

In [ ]:
# Checking for any strange values
df.condition.value_counts()

In [ ]:
# Checking for any strange values
df.grade.value_counts()

In [ ]:
# Checking for any strange values
df.sqft_above.value_counts()[:20]

> There are question mark values for sqft_basement. I decided to fill them with zeros.

In [ ]:
# Checking for any strange values
df.sqft_basement.value_counts()[:20]

In [ ]:
# Replacing the ? entries with 0
df.sqft_basement = df.sqft_basement.map(lambda x: 
                                        int(float(x.replace('?', '0'))))

In [ ]:
# Setting the datatype to be an integer
df.sqft_basement = df.sqft_basement.astype('int64')

In [ ]:
# Checking for any strange values again
df.sqft_basement.value_counts()[:20]

In [ ]:
# Checking for any strange values
df.yr_built.value_counts()[:20]

In [ ]:
# Checking for any strange values
df.zipcode.value_counts()[:20]

In [ ]:
# Checking for any strange values
df.lat.value_counts()

In [ ]:
# Checking for any strange values
df.long.value_counts()

In [ ]:
df.sqft_living15.value_counts()[:20]

In [ ]:
df.sqft_lot15.value_counts()[:20]

In [ ]:
# Final check that all the data types are good
df.info()

## Adding Columns

There are some columns I want to add based on the data I have in other columns that I think may be of more use to my model than the current columns. For example the sqft_basement column has mostly 0 entries. It may be more useful to have a column that indicates whether a basement exists or not. Also with dates, it may be useful to have columns based on month or season sold.

In [ ]:
# Spliting the month out of the date column into a new column of its own
df['month_sold'] = df['date'].map(lambda x: x.replace('/', ' ').split()[0])

In [ ]:
# Casting the new column to be an integer
df['month_sold'] = df['month_sold'].astype('int64')

In [ ]:
# Checking the values of the new column
df.month_sold.value_counts()

In [ ]:
# Function that takes the day from the date column 
# and assigns a week of the month

def week_of_month(x):
#     Splitting the day out from the date
    day = int(x.replace('/', ' ').split()[1])
    
#     Assigning the week based on the day
    if day < 8:
        week = 1
    elif day < 15 and day > 7:
        week = 2
    elif day < 22 and day > 14:
        week = 3
    elif day < 29 and day > 21:
        week = 4
    else:
        week = 5
        
    return week

In [ ]:
# Adding the new column by applying the function above
df['week_sold'] = df['date'].map(week_of_month)

In [ ]:
# Checking the values of the new column
df.week_sold.value_counts()

In [ ]:
# Function that takes the month sold and assigns the month to a season

def season_sold(x):
    
#     Assigning season based on month
    if x > 2 and x < 6:
        season = 1
    elif x > 5 and x < 9:
        season = 2
    elif x > 8 and x < 12:
        season = 3
    else:
        season = 4
        
    return season

In [ ]:
# Creating the new column by applying the function above
df['season_sold'] = df['month_sold'].map(season_sold)

In [ ]:
# Checking the values of the new column
df.season_sold.value_counts()

In [ ]:
# Creating new renovated column
df['was_renovated'] = df['yr_renovated'].map(lambda x: x!=0)

In [ ]:
# Casting the new column to be an integer
df['was_renovated'] = df['was_renovated'].astype('int64')

In [ ]:
# Checking the values of the new column
df.was_renovated.value_counts()

In [ ]:
# Creating new basement column
df['has_basement'] = df['sqft_basement'].map(lambda x: x!=0 )

In [ ]:
# Casting the new column to be an integer
df['has_basement'] = df['has_basement'].astype('int64')

In [ ]:
# Checking the values of the new column
df.has_basement.value_counts()

In [ ]:
# Dropped the date column since the values will be hard to model with
df.drop(columns='date', inplace=True)

In [ ]:
# Checking all my columns at data types
df.info()

# Exploring the data

## EDA Question 1: How does location affect house prices, sizes, and other metrics? 

I was curious to do some exploration with the latitude and longitude to map the houses and look at various factors like price, home square footage, and lot square footage. The visuals here were created using my cleaned data loaded into Tableau Public. The full images can be viewed and downloaded from https://public.tableau.com/profile/vivienne4370

In [ ]:
# Saving the file to use
# df.to_csv(r'C:\Users\drudi\DataScience\Module02\FinalProject\cleaned_data.csv')

### Location and price

<img src="Home Prices By Location.png">

<img src="Median Home Price By Zipcode.png">

> These maps show that the areas around Seattle and Bellevue are the most expensive. I am surprised that Bellevue is actually more expensive than Seattle. In general though the trend is that the closer to the urban center, then the more expensive. Which is a trend I expected.

### Location and squarefootage

<img src="Squarefoot Lot By Location.png">

<img src="Squarefoot Living Space By Location.png">

> As expected the lot size increases the further out you get from the urban center. However, I am surprised to see that in the Bellevue area there are homes with some larger lots. Home squarefootage also surprises me that there are some very large homes in the urban center. I would expect the homes to have more of a trend of being larger the further you get out from the city. That is somewhat the case, but not as dramatically as I would expect.

### Location and home age

<img src="Year Built By Location.png">

> This visualization shows the clear trend of the oldest homes being in the urban center and how houses were built out from there over time. It's interesting to see the smattering of newer homes in the urban center and pockets of older homes on the outskirts. It would be interesting to know more of the history of these old homes.

In [ ]:
df.describe()

## Checking the distribution of prices

In [ ]:
# Setting the figure and plotting
fig, ax = plt.subplots(figsize=(10,6))
sns.distplot(df['price'], bins='auto')

# Adjusting the money ticks 
fmt_money = '${x:,.0f}'
tick_money = mtick.StrMethodFormatter(fmt_money)
ax.xaxis.set_major_formatter(tick_money)
plt.xticks(rotation=45)

# Setting the title and labels
ax.set_xlabel('Price', fontsize=15)
ax.set_title('House Prices', fontsize=20);

>There is a large tail to the distribution of prices which is to be expected since there are of course a few houses that are much more expensive than the majority of houses. I will address all the outliers at a later point, but it's good to see that there looks to still be good normality in the prices despite the long tail.

## Checking linearity, normality distribution, and choosing categorical columns

In [ ]:
# Creating a function to plot a single feature against price
def explore_plot(df, col, target='price'):
    
#     Plotting the graph and setting the regression line to be red
    g = sns.jointplot(data=df, x=col, y=target, kind='reg', height=6, 
                      joint_kws={'line_kws':{'color':'red'}})

#     Setting the title
    plt.suptitle(f'{col} and {target}', fontsize=20, y=1.05)
    return g

In [ ]:
# Looping through the columns to feed each into the function above
cols = list(df.columns)
for col in cols:
    explore_plot(df, col)

> Based on these graphs, I decided to make condition, zipcode, month_sold, week_sold, and season_sold categorical features. I want to turn yr_built into a category as well since there is no linearity in the relationship. Others have a very weak linear relationship, but I will leave them for now with my first model and judge them by the p-values later. There are many features that also do not have normality or have a lot of outliers which I will address for other models. For this first model I want to leave the data as is to better judge improvements.

In [ ]:
categories = ['condition', 'zipcode','month_sold', 'week_sold', 'season_sold']
for col in categories:
    df[col] = df[col].astype('category')

## Making year built categorical

In [ ]:
df.yr_built.describe()

In [ ]:
# Funcation that takes the year built and puts it into a bucket for decade

def built_decades(x):
#     Assigning the decade built

    if x < 1910:
        decade = 0
    elif x < 1920 and x > 1909:
        decade = 1
    elif x < 1930 and x > 1919:
        decade = 2
    elif x < 1940 and x > 1929:
        decade = 3
    elif x < 1950 and x > 1939:
        decade = 4
    elif x < 1960 and x > 1949:
        decade = 5
    elif x < 1970 and x > 1959:
        decade = 6
    elif x < 1980 and x > 1969:
        decade = 7
    elif x < 1990 and x > 1979:
        decade = 8
    elif x < 2000 and x > 1989:
        decade = 9
    elif x < 2010 and x > 1999:
        decade = 10
    else:
        decade = 11
    return decade

In [ ]:
# Mapping the function to the column to change the values
df['yr_built'] = df['yr_built'].map(built_decades)

In [ ]:
df['yr_built'] = df['yr_built'].astype('category')

In [ ]:
# Checking that the values changed
df.yr_built.value_counts()

In [ ]:
df.info()

## EDA Question 2: What are the average home prices for the categorical features?

Now that I have my categorical features picked, I wanted to look more closely at how they affect home prices. I decided to plot them all to see what kinds of interesting trends I would find.

In [ ]:
# Function that plots each of the categorical features
def plot_categories(df):
    
#     Loops through the category data types
    for col in df.select_dtypes('category'):
        
#       Creating the figure and setting the fig size
        fig, ax = plt.subplots(figsize=(10,6))
    
#       Plotting each column at a time
        sns.barplot(x=col, y='price', data=df, palette="husl")
    
#       Setting the title and formatting the x-ticks for better visibility
        ax.set_title(f"{col} vs Price", fontsize=30)
        plt.xticks(rotation=45)
        plt.show()

In [ ]:
plot_categories(df)

> From these graphs there are some interesting insights:
* Condition shows that there is a division between a category of 1 or 2 and a category of 3, 4, or 5. One would expect a more linear relationship here, but it seems that it has created two buckets of lower and higher condition.
* Year built is interesting that it shows a drop in price starting for houses in the 40s. I expected this to be a linear relationship but from this graph I would guess that many of the very oldest homes are desired for their uniqueness and perhaps certain craftsmanship that is iconic to the era. Then starting for houses built in the 40s, houses are not seen as iconic and charming anymore, and the newer the house the better. 
* The zipcode graph shows pretty much the same thing as the visual from before. There are huge differences in price based on zipcode.
* I was expecting to see a greater difference among the graphs for month, week, and season sold. Winter has a slightly lower average price and February is the worst month. I was also expecting to see a bigger trend with the weeks that maybe it was better to sell a house and the beginning or very end of the month for example. It's interesting to see that the time of selling doesn't make as big of a difference as I thought it would.

# Model 1

## Preparing the data for modeling

I made a dataframe specifically for modeling to leave my original clean dataframe untouched. I decided to drop latitude and longitude for modeling as zipcode will be a more meaningful location feature.

In [ ]:
# Creating new modeling dataframe
model1_df = df.copy()

# Dropping columns
model1_df.drop(columns=['lat', 'long'], inplace=True)

In [ ]:
# Checking the new dataframe
model1_df.head()

In [ ]:
# Checking the new dataframe
model1_df.info()

## Writing the modeling function

In [ ]:
# Function that will run my model and some diagnostics

def make_model(df, target='price', test_size=0.25, cv=20):

#     Definging X and y
    X = df.drop([target], axis=1)
    y = df[[target]]
    
#     Performing a train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=test_size, 
                                                        random_state=123)
    
#     Creating dataframes with the split data
    df_train = pd.concat([y_train, X_train], axis=1)
    df_test = pd.concat([y_test, X_test], axis=1)
    
#     Pulling out the categorical columns
    cat_cols = df_train.select_dtypes('category').columns
    
#     Writing the formula to feed into my model
    features = '+'.join(df_train.drop(columns=target).columns)
    
#     Looping through the categoricals to format the formula correctly
    for col in cat_cols:
        features = features.replace(col,f'C({col})')
        
#     The completed formula to feed in to the model    
    formula = target + '~' + features
    
#     Putting my training data through the model
    model = smf.ols(formula, df_train).fit()
    
#     Plotting a qq plot of the residuals to check for normality
    fig, axes = plt.subplots(ncols=2, figsize=(20, 5))
    sm.graphics.qqplot(model.resid, fit=True, line='45', ax=axes[0])
    axes[0].set_title('QQ Plot Normality Check', fontsize=20)
    
#     Plotting the residuals to check for homoscedasticity 
    ax=axes[1]
    ax.scatter(df_train['price'], model.resid)
    ax.axhline(0, color='red')
    axes[1].set_title('Homoscedasticity Check', fontsize=20)
    
#   This will make both plots appear
    plt.show();

#     Getting the predicted y values from the model
    y_predicted = model.predict(X_test)
    
#     Plotting a scatterplot of the predicted vs actual test data prices 
#     to visually inspect how different they are
    plt.figure(figsize=(20,5))
    
#     Plotting the first 200 entries of the predicted and actual prices
    g = sns.scatterplot(range(len(y_predicted[:200])), y_predicted[:200], 
                        label='Predicted Prices')
    g = sns.scatterplot(range(len(y_test[:200])), y_test.price[:200], 
                        label='Actual Prices')

#     Setting the title, labels, and legend of the plot
    plt.title('Comparison of predicted vs actual price', 
              fontdict={'fontsize':20})
    plt.xlabel('Values')
    plt.ylabel('Prices')
    plt.legend()
    plt.show();
    
#   Getting the r2 for the test data to compare to the train data 
    r2_test = r2_score(y_test, y_predicted)
    print('Model test data R2 score:', r2_test)
    
#   Doing a cross validation k-fold
    cv_result = np.mean(cross_val_score(linreg, X, y, cv=cv, 
                                        scoring='neg_mean_squared_error'))
    print('K-fold cross validation negative MSE:', cv_result)
    
#   Finally, displaying the model summary
    display(model.summary())
    
#   The model is the object returned so I can perform different functions on it
    return model

## Results

In [ ]:
model1 = make_model(model1_df)

> Insights for this model:
* The r2 of the model is pretty high but the r2 of the test data is very off from the model which shows that it's not a very good model.
* The qq plot and the homoscedasticity plot are both showing that the residuals do not have normality or homoscedasticity. 
* The scale on the plot of predicted vs actual prices is large and many of the values are hanging out at the bottom of the graph being reasonably close, but there are a few that are wildly off. 
* The cross validation score is negative MSE so the higher the number, the better. It is a very large negative number so I expect to see that number improve in future models.
* Some p values are very significant, some are very not significant.
* Overall this is just a starting point to compare to see how much I can improve various metrics while sparing model accuracy.

# Model 2

## Removing outliers

For this model I'm only going to remove the outliers to see how that improves my model. Originally I experimented with removing outliers using z-scores, cook's distance, and interquartile range. The interquartile range improved my model the most so that is the method I stuck with.

In [ ]:
# Function to find the outliers according to the IQR method
def iqr_outliers(data):
    
#     Defining the quartiles and finding the IQR
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1
    
#     Defining the threshold and comparing the data against the threshold
    threshold = iqr * 1.5
    outliers = (data < q1 - threshold) | (data > q3 + threshold)
    
#     Making a series out of the outliers
    outliers = pd.Series(outliers, index=data.index)
        
    return outliers

In [ ]:
# Function to add the outliers for each column as a new column in the dataframe
def add_outliers_column(df, columns, verbose=True):
    
#     Makes a new dataframe to leave the previous unedited
    new_df = df.copy()
    
#     Iterates through columns
    for col in columns:
        
#         References the previous function to find the outliers
        outliers = iqr_outliers(new_df[col])
        
#         Printing how many outliers were found in each column
        if verbose:
            print(f'{outliers.sum()} outliers found in {col}')
    
#         Adds the outliers as a new column
        new_df[f'{col}_outliers'] = outliers
        
    return new_df

In [ ]:
# Feeding in the numerical columns for identifying outliers
num_cols = list(model1_df.select_dtypes('number').columns)

# Saving the new dataframe with outlier columns added
(f'{outliers.sum()} outliers found')(f'{outliers.sum()} outliers found') = add_outliers_column(model1_df, num_cols)

In [ ]:
# Checking out my new dataframe
model2_df.head()

> I am going to drop the waterfront and was_renovated outlier columns since the outliers identified were all of the ones that are waterfront or were renovated. I don't want to lose those features completely. I'm also going to drop the outlier column for yr_renovated as the idea of outliers in that column doesn't make any sense. 

In [ ]:
# Dropping the selected new outlier columns
model2_df.drop(columns=['waterfront_outliers', 'yr_renovated_outliers', 
                        'was_renovated_outliers'], inplace=True)

In [ ]:
# Checking my dataframe before outlier removal
model2_df.head()

In [ ]:
# Creating a list of the outlier columns
outlier_cols = []
for col in model2_df.columns:
    if 'outliers' in col:
        outlier_cols.append(col)
outlier_cols

In [ ]:
# Filtering the outliers out of my dataframe and returning only the entries
# that are not outliers for any of the features
for col in outlier_cols:
    model2_df = model2_df[(model2_df[col]==False)]
model2_df.shape

In [ ]:
# Dropping the outlier columns that were added
for col in outlier_cols:
    model2_df.drop(columns=[col], axis=1, inplace=True)

In [ ]:
# Checking my dataframe for the right data types and columns
model2_df.info()

## Checking linearity and normality after outlier removal

In [ ]:
# Setting the figure and plotting
fig, ax = plt.subplots(figsize=(10,6))
sns.distplot(model2_df['price'], bins='auto')

# Adjusting the money ticks 
fmt_money = '${x:,.0f}'
tick_money = mtick.StrMethodFormatter(fmt_money)
ax.xaxis.set_major_formatter(tick_money)
plt.xticks(rotation=45)

# Setting the title and labels
ax.set_xlabel('Price', fontsize=15)
ax.set_title('House Prices', fontsize=20);

> With the outliers removed from price there is a much more normal looking distribution than before.

In [ ]:
# Looping through the columns to feed each into the function above
cols = list(model2_df.columns)
for col in cols:
    explore_plot(model2_df, col)

> Many of the features are more normally distributed with less dramatic tails. I decided to remove both sqft_lot and sqft_lot15 since there is not a good linear relationship there. I also decided to remove yr_renovated and sqft_basement as both of those columns are so full of zeros. Insteaqd I will rely on the was_renovated and has_basement columns to account for renovations and basements. I am hoping that removing these will improve my residual normality and homoscedasticity.

In [ ]:
model2_df.drop(columns=['sqft_lot', 'sqft_lot15', 'yr_renovated', 'sqft_basement'], inplace=True)

In [ ]:
model2_df.head()

## Results

In [ ]:
model2 = make_model(model2_df)

> Insights for this model:
* The r2 score has gone up by over 2% points and the r2 for the test data is much closer to the training data this time. 
* The residuals are greatly improved being more normally distributed and being more homoscedastic. There is still room for improvement on the residuals.
* The scale on the comparison of the actual and predicted prices has gone way down so the two are much closer together. 
* The cross validation score has gone up by a lot.
* Most p-values look significant but I have some wildly large p-values to address.
* Overall, a great step of improvement. Let's see how I can improve it more.

# Model 3

## Removing low p-values

My next step is to address the features with low p-values. Any features that are categorical will be left in the modeling if the majority of the categories have a significant p-value. Likewise, the whole category will be removed if the majority have non-significant p-values.

In [ ]:
# Function to identify the insignificant p-values
def bad_pvalues(model, verbose=True):
    
#     Pulling out the p-values and identifying the ones above .05
    pvalues = model.pvalues
    bad_features = pvalues[pvalues > .05]
    
#     Excluding the intercept in case it has a high p-value
    if 'Intercept' in bad_features:
        bad_features.remove('Intercept')
        
#      Printing a statement of the bad p-values
    if verbose:
        print(f'{len(bad_features)} bad p-values to be reviewed:\n' + f'{bad_features}')
    return bad_features

In [ ]:
model2_bad_pvals = bad_pvalues(model2)

> From this list I am going to remove week sold as none of the categories seem to be very significant. I am going to leave yr_built, month_sold, and zipcode alone as the majority of those categories are significant.

In [ ]:
model3_df = model2_df.copy()
model3_df.drop(columns=(['week_sold']), inplace=True)
model3_df.shape

## Results

In [ ]:
model3 = make_model(model3_df)

> Insights for this model:
* I was hoping that my residuals would improve but they look the same as before.
* The r2 is the same as before
* Overall, removing the low p-value features didn't have very much impact on my model.

# Model 4

My approach for this next model iteration is to address multicolinearity and VIF. I followed a guideline of about a 0.70 threshold for multicolinearity and 6 for VIF.

## Multicolinearity

In [ ]:
# Creating the dataframe for model iteration 4
model4_df = model3_df.copy()

In [ ]:
# Writing a function to create a correlation heat map and chart
def multicol_plot(df):
    
#   Creating a temporary new dataframe
    new_df = df.copy()
    
#   Converting the category types to int so they will show up in the heatmap
    categories = new_df.select_dtypes('category')
    for col in categories:
        new_df[col]= new_df[col].astype('int64')  
        
#   Generating the correlation chart 
    corr = abs(new_df.corr())
    
#   Creating a mask that will eliminate redundant values in the heatmap
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True
    
#   Plotting the figure and applying the mask
    fig = plt.figure()
    fig.set_size_inches(12,12)
    sns.heatmap(corr, annot=True, mask=mask)
    plt.show();
    
    display(corr)
    return corr

In [ ]:
# Running the function
model4_corr = multicol_plot(model4_df)

> Interpretation of the results:
* Squarefoot above and squarefoot living are highly correlated. I am going to keep squarefoot living since the overall squarefootage of a home is a more meaningful metric.
* Squarefoot living and squarefoot living15 are also highly correlated but again I think that the squarefootage of the home is the more meaningful metric so I will eliminate the other.

In [ ]:
# Dropping selected columns
model4_df.drop(columns=['sqft_above', 'sqft_living15'], inplace=True)

## Variance Inflation Factor

In [ ]:
# Writing a function to get vif scores for features
def vif_results(df, target='price'):
    
#   Dropping the target and adding a constant
    new_df = df.drop(columns=target, axis=1)
    new_df = sm.add_constant(new_df)
    
#   Turning the category data types to integers so they show up in the VIF
    categories = new_df.select_dtypes('category')
    for col in categories:
        new_df[col]= new_df[col].astype('int64')    

#   Creating an empty list to append to  
    vif_list = []
#   Iterating through the columns of the dataframe
    for x in range(new_df.shape[1]):
        
#       Running the vif for each column and adding it to the empty list
        vif = variance_inflation_factor(new_df.values, x)
        vif_list.append(vif)
        
#   Creating a series for the columns and the vif list
    results = pd.Series(dict(zip(new_df.columns, vif_list)))
    print(results)
    
#   Identifying features that are above the threshold
    threshold = 6
    bad_columns = list(results[results > threshold].index)
    
#   Removing the constant from being included in the bad features
    if 'const' in bad_columns:
        bad_columns.remove('const')
        
    return bad_columns

In [ ]:
vif_columns = vif_results(model4_df)
vif_columns

> There are no features above the threshold so I am not going to remove any more features.

## Results

In [ ]:
model4 = make_model(model4_df)

> Insights for this model:
* The the graphs for the residuals look the same and have not improved.
* The r2 score and cross validation score actually got a little bit worse.

# Model 5

## Log transforming price

My next tactic is going to be to log transform the price column to try to achieve better normality and homoscedasticity with my residuals.

In [ ]:
# Creating a new dataframe for this model iteration
model5_df = model4_df.copy()

In [ ]:
# Log transforming the price column
model5_df['price'] = model5_df['price'].map(lambda x: np.log(x))

## Checking linearity and normality after log transforming price

In [ ]:
# Setting the figure and plotting
fig, ax = plt.subplots(figsize=(10,6))
sns.distplot(model5_df['price'], bins='auto')

# Setting the title and labels
ax.set_xlabel('Price', fontsize=15)
ax.set_title('House Prices', fontsize=20);

In [ ]:
# Checking the distributions and linearity for my features again
cols = list(model5_df.columns)
for col in cols:
    explore_plot(model5_df, col)

## Results

In [ ]:
model5 = make_model(model5_df)

> Insights for this model:
* The residuals for the qq plot have improved for the tail on the higher end, but the lower end have worsened. The bottom of the scale was -6 in the last model and now it is -8.
* It's hard to judge if homoscedasticity has improved as the scale has also changed but there is still a little bit of a cone shape.
* It is hard to tell if the comparison of the predicted and actual prices improved because the scale changed.
* The r2 score went up by about .02 which is a good improvement.
* The cross validation score greatly improved and is a much higher number than before.

# Model 6

## Checking p-values

In [ ]:
# Checking for high p-values from this model
model5_bad_pvals = bad_pvalues(model5)

> Bedrooms now has an insignificant p value after log transforming price and will need to be removed.

In [ ]:
# Removing the bedrooms column from my next dataframe iteration
model6_df = model5_df.copy()
model6_df.drop(columns=['bedrooms'], inplace=True)

## Multicolinearity

In [ ]:
# Checking if columns need to be addressed for multicolinearity again
model6_corr = multicol_plot(model6_df)

> Nothing is above .70 so there is nothing to remove here.

## VIF

In [ ]:
vif_columns = vif_results(model6_df)
vif_columns

> No high VIF columns either.

## Results

In [ ]:
model6 = make_model(model6_df)

## Last p-value check

In [ ]:
# Checking p-values
model6_bad_pvals = bad_pvalues(model6)

> There are no features to removed based on p-values as these are all categories where the majority of the categories are significant.

> **I am going to use this model iteration for my final model as I have done all I can to improve the model and the residuals while still keeping interpretability.**

# EDA Question 3: Would the residuals improve from log transforming all the non categorical columns?

Since I had such a hard time seeing improvement in the residuals for this project, I wanted to see what would the improvement look like if I log transformed all of the non categorical columns. Once all the columns are log transformed then I lose the ability to interpret the model so it's not practical for this project, but I wanted to see what the model would look like.

In [ ]:
# Making a copy of the dataframe for this iteration
model7_df = model6_df.copy()

In [ ]:
# Grabbing the columns appropriate to transform
num_cols = ['bathrooms', 'sqft_living', 'floors', 'grade']

# Iterating through the number columns and log transforming them
for col in num_cols:
    model7_df[col] = model7_df[col].map(lambda x: np.log(x))

In [ ]:
model7_df.describe()

## Results

In [ ]:
model7 = make_model(model7_df)

> Insights from this model:
* I am surprised that the model residuals did not improve. There was not a lot of change for better or worse.
* I am also surprised that there was not a change in the r2 score or the cross validation score.
* Overall the model quality is pretty much the same as the previous model but with a lot less interpretability.


In [ ]:
# Checking for insignificant p-values
model7_bad_pvals = bad_pvalues(model7)

> There are more insignificant p-values in this model than in model 6 but still nothing that would justify removing. Again, I am surprised that there isn't a more dramatic difference. 